In [98]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [106]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
start = time.time()


meta = pd.read_csv("./AutoKaggle - Metadata.csv", error_bad_lines=False)
arrOfRows = [64,360,239]
row = 237
train = ''
def preprocessing(row):
    find_row = meta.loc[row]
    train = ''
    test = None
    check_test = True
    train_X = ''
    train_Y = ''
    test_X = None
    if meta['name'].loc[row] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv")
        check_test = False
    elif meta['name'].loc[row] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
    elif meta['name'].loc[row] == 'uciml/sms-spam-collection-dataset':
        train = pd.read_csv("./uciml_sms-spam-collection-dataset/data/spam.csv", error_bad_lines=False)
        check_test = False
        
    
    if check_test:
        test = test.dropna()
    if meta['name'].loc[row] == 'uciml/sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    else:
        train = train.dropna()
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_test:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))

        targetName = find_row['targetName']
        train_Y = train[targetName]
        train_X = train.drop(columns=targetName)
        if check_test:
            test_X = test
            return train_X, train_Y, test_X
        else:
            return train_X, train_Y, None


preprocessing(row)


b'Skipping line 167: expected 102 fields, saw 164\nSkipping line 203: expected 102 fields, saw 108\nSkipping line 282: expected 102 fields, saw 105\nSkipping line 286: expected 102 fields, saw 241\nSkipping line 290: expected 102 fields, saw 928\nSkipping line 341: expected 102 fields, saw 171\nSkipping line 342: expected 102 fields, saw 119\nSkipping line 361: expected 102 fields, saw 106\nSkipping line 392: expected 102 fields, saw 531\nSkipping line 395: expected 102 fields, saw 169\nSkipping line 409: expected 102 fields, saw 120\nSkipping line 417: expected 102 fields, saw 201\nSkipping line 418: expected 102 fields, saw 167\nSkipping line 426: expected 102 fields, saw 115\nSkipping line 431: expected 102 fields, saw 162\nSkipping line 434: expected 102 fields, saw 108\n'
b'Skipping line 55: expected 5 fields, saw 6\nSkipping line 710: expected 5 fields, saw 7\nSkipping line 896: expected 5 fields, saw 8\nSkipping line 1032: expected 5 fields, saw 10\nSkipping line 1115: expected 

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [107]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test):
    if meta['name'].loc[row] == 'uciml/sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    elif meta['name'].loc[row] == 'mercedes-benz-greener-manufacturing':
        if type(meta["function call feature extraction"].loc[row]) is not str:
            print('not func')
            return X_train,X_test
        extraction_function_calls = str(meta["function call feature extraction"].loc[row])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(meta["function parameters feature extraction"].loc[row])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [108]:
train = preprocessing(row)
feature_extraction(row, train, None)

b'Skipping line 55: expected 5 fields, saw 6\nSkipping line 710: expected 5 fields, saw 7\nSkipping line 896: expected 5 fields, saw 8\nSkipping line 1032: expected 5 fields, saw 10\nSkipping line 1115: expected 5 fields, saw 7\nSkipping line 1268: expected 5 fields, saw 6\nSkipping line 1367: expected 5 fields, saw 13\nSkipping line 1406: expected 5 fields, saw 7\nSkipping line 1534: expected 5 fields, saw 7\nSkipping line 2343: expected 5 fields, saw 7\nSkipping line 2380: expected 5 fields, saw 12\nSkipping line 2944: expected 5 fields, saw 7\nSkipping line 3518: expected 5 fields, saw 7\nSkipping line 3610: expected 5 fields, saw 8\nSkipping line 3726: expected 5 fields, saw 6\nSkipping line 3745: expected 5 fields, saw 6\nSkipping line 3839: expected 5 fields, saw 6\nSkipping line 3849: expected 5 fields, saw 6\nSkipping line 3925: expected 5 fields, saw 6\nSkipping line 4100: expected 5 fields, saw 8\nSkipping line 4412: expected 5 fields, saw 12\nSkipping line 4622: expected 5 f

(<3841x269 sparse matrix of type '<class 'numpy.float64'>'
 	with 3801 stored elements in Compressed Sparse Row format>,
 <1647x269 sparse matrix of type '<class 'numpy.float64'>'
 	with 1629 stored elements in Compressed Sparse Row format>,
 4656     ham
 5328     ham
 2572     ham
 2950     ham
 4352     ham
 576      ham
 3925     ham
 841      ham
 1479     ham
 682      ham
 1299     ham
 776     spam
 3010     ham
 830      ham
 358      ham
 1459     ham
 5133     ham
 4536     ham
 177      ham
 4714     ham
 4893    spam
 2805     ham
 4518     ham
 1142     ham
 1427    spam
 1585    spam
 4736     ham
 1429     ham
 179      ham
 702      ham
         ... 
 3095     ham
 3634     ham
 2656     ham
 270      ham
 2846     ham
 1099    spam
 502      ham
 1556    spam
 263      ham
 4369     ham
 3834     ham
 3757     ham
 2558     ham
 5415     ham
 4724    spam
 1489     ham
 3043     ham
 3642     ham
 3709     ham
 4726     ham
 5150     ham
 3594     ham
 1635    spam
 2

In [109]:
def feature_selection():
    pass

In [110]:
def estimation(row,X_train,X_test,Y_train, Y_test):
    if meta['name'].loc[row] == 'uciml/sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    else:
        estimation_function_calls = meta["function calls estimation"].loc[row]
        estimation_function_calls = estimation_function_calls.split(",")
        print(type(meta["function parameters estimation"].loc[row]))
        print(meta["function parameters estimation"].loc[row])
        estimation_function_param = eval(meta["function parameters estimation"].loc[row])

        print(len(estimation_function_calls))
        if len(estimation_function_calls) == 1:
            l_str = estimation_function_calls[0].split("(")
            l_str.insert(1,'('+estimation_function_param[0])
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'estimator' + '=' + str_call
            exec(str_call,globals(),globals())
    #         estimator.fit(X_train,Y_train)
            print(cross_val_score(estimator, X_train, Y_train, cv=3, n_jobs=8))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                str2 = extraction_funtion_param
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)

In [111]:
if meta['name'].loc[row] == 'uciml/sms-spam-collection-dataset':
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)

b'Skipping line 55: expected 5 fields, saw 6\nSkipping line 710: expected 5 fields, saw 7\nSkipping line 896: expected 5 fields, saw 8\nSkipping line 1032: expected 5 fields, saw 10\nSkipping line 1115: expected 5 fields, saw 7\nSkipping line 1268: expected 5 fields, saw 6\nSkipping line 1367: expected 5 fields, saw 13\nSkipping line 1406: expected 5 fields, saw 7\nSkipping line 1534: expected 5 fields, saw 7\nSkipping line 2343: expected 5 fields, saw 7\nSkipping line 2380: expected 5 fields, saw 12\nSkipping line 2944: expected 5 fields, saw 7\nSkipping line 3518: expected 5 fields, saw 7\nSkipping line 3610: expected 5 fields, saw 8\nSkipping line 3726: expected 5 fields, saw 6\nSkipping line 3745: expected 5 fields, saw 6\nSkipping line 3839: expected 5 fields, saw 6\nSkipping line 3849: expected 5 fields, saw 6\nSkipping line 3925: expected 5 fields, saw 6\nSkipping line 4100: expected 5 fields, saw 8\nSkipping line 4412: expected 5 fields, saw 12\nSkipping line 4622: expected 5 f

0.8809957498482088


In [105]:
def postprocessing(estimators,stack):
    pass

In [ ]:
end = time.time()
print(end - start)